<a href="https://colab.research.google.com/github/sadra-barikbin/persian-information-retrieval-example/blob/main/Persian-IR-example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [ ]:
!pip install hazm transformers ir_measures
!pip install -q clean-text[gpl]

<div dir='rtl'>
    ابتدا کتاب‌خانه‌های لازم را فرا می‌خوانیم
</div>

In [75]:
import torch
import yaml
import hazm
import tqdm
import numpy as np
import pandas as pd
import ir_measures as IRm
import tensorflow
from tempfile import mkdtemp
from typing import List, Tuple
from pathlib import Path
from sklearn.metrics import make_scorer, average_precision_score
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
from sklearn.pipeline import Pipeline
from sklearn.model_selection import ParameterGrid
from sklearn.base import BaseEstimator, TransformerMixin
from transformers import AutoTokenizer, AutoModelForMaskedLM, AutoConfig, AutoTokenizer, AutoModel, TFAutoModel

# Loading & Preparing Data

## Corpus

In [ ]:
!wget https://github.com/language-ml/2-LM-embedding-projects/raw/main/problem3/doc_collection.zip

In [ ]:
!unzip doc_collection.zip -d dataset

In [ ]:
!cat IR_dataset/1000.txt

<div dir='rtl'>
    در این بخش مجموعه دادگان دانلود و استخراج می‌شود و سپس از روی آن
    Corpus
    برای ساخت مدل‌های زبانی ایجاد می‌کنیم
</div>

In [7]:
corpus = [(int(path.stem), path.open().read()) for path in Path('dataset/IR_dataset').iterdir()]
corpus = pd.DataFrame(corpus, columns=['docId','text']).set_index('docId').sort_index()

In [ ]:
corpus.head()

,text
docId,
0,برخی از هواداران مصدق یا اعضای جبهه ملی که در ...
1,جبهه ملی ایران که به اختصار جبهه ملی نیز خواند...
2,سرلشکر زاهدی در سال ۱۳۲۸ و پس از آن‌که دخالت‌ه...
3,نمایندگان طرفدار مصدق در حمایت از ابقای دولت و...
4,نمایندگان طرفدار مصدق در حمایت از ابقای دولت و...


## Qrels

<div dir='rtl'>
    در این بخش فایل کوئری‌ها را دانلود می‌کنیم و از روی فایل آن دیتافریمی می‌سازیم که کوئری و داک بازیابی شده با سطح نزدیکی را نمایش دهد
</div>

In [ ]:
!wget https://raw.githubusercontent.com/language-ml/2-LM-embedding-projects/main/problem3/evaluation_IR.yml -P dataset

In [10]:
query_raw_data = yaml.safe_load(open('dataset/evaluation_IR.yml'))

In [11]:
query = pd.Series(query_raw_data.keys())
qrels = [{'query_id':idx, 'doc_id':d,
          'relevance':3} for idx,q in query.to_dict().items() for d in query_raw_data[q]['similar_high']]
qrels.extend([{'query_id':idx, 'doc_id':d,
          'relevance':2} for idx,q in query.to_dict().items() for d in query_raw_data[q]['similar_med']])
qrels.extend([{'query_id':idx, 'doc_id':d,
          'relevance':1} for idx,q in query.to_dict().items() for d in query_raw_data[q]['similar_low']])
qrels.extend([{'query_id':idx, 'doc_id':query_raw_data[q]['relevant'][0],
          'relevance':4} for idx,q in query.to_dict().items()])
qrels = pd.DataFrame(qrels)

In [ ]:
query[147],query_raw_data[query[147]]

('گرجستان  تاریخ',
 {'relevant': [388],
  'similar_high': [389, 390, 391, 392, 393, 394],
  'similar_low': [404, 405, 406, 407, 408, 409, 410, 411, 412, 413],
  'similar_med': [395, 364, 396, 397, 398, 399, 400, 401, 402, 403]})

In [12]:
qrels.sample(n=5).reset_index(drop=True)

,query_id,doc_id,relevance
0,134,88,1
1,147,393,3
2,149,1512,3
3,129,1084,2
4,125,451,1


## Normaliztion

<div dir='rtl'>
    با استفاده از نرمال‌ساز کتابخانه هضم، فایل‌های خود را نرمال می‌کنیم که در فرآیند ساخت مدل زبانی بهتر عمل کنیم
</div>

In [13]:
normalize = hazm.Normalizer().normalize
corpus.text = corpus.text.transform(normalize)
query = query.transform(normalize)

# Embedding the documents

## Method 1 : Tfidf

<div dir='rtl'>
    برای ساخت بردارها بر اساس
    Tf-Idf
    از کتاب‌خانه
    sklearn
    استفاده می‌کنیم
</div>

In [14]:
vectorizer = TfidfVectorizer()
vectorizer.fit(corpus.text)

TfidfVectorizer()

## Method 2 : ParsBert

<div dir='rtl'>
    برای استفاده از برت، از
    ParsBert
    استفاده می‌کنیم.
</div>

In [ ]:
model_name_or_path = "HooshvareLab/bert-fa-zwnj-base"
config = AutoConfig.from_pretrained(model_name_or_path)
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)

In [ ]:
model = AutoModel.from_pretrained(model_name_or_path, from_tf = True)
model = model.cuda()

All TF 2.0 model weights were used when initializing BertModel.

All the weights of BertModel were initialized from the TF 2.0 model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use BertModel for predictions without further training.


In [ ]:
text = "ما در قرن ۲۱ زندگی می‌کنیم" 
encoding = tokenizer.encode_plus(
      text,
      add_special_tokens=True, # Add '[CLS]' and '[SEP]'
      return_token_type_ids=False,
      max_length = 500,
      truncation=True,
      return_attention_mask=True,
      return_tensors='pt',  # Return PyTorch tensors
    )
out = model(
            input_ids = encoding['input_ids'].cuda(), 
            attention_mask= encoding['attention_mask'].cuda())
out['pooler_output'][0]

<div dir='rtl'>
    ابتدا تنظیمات لازم را برای استفاده از مدل از پیش آموزش داده شده تنظیم می‌کنیم.
    سپس از آنجا که طول داک‌ها بعضا بیشتر از ۵۰۰ کلمه هستند،
    هر فایل را به پنجره‌های ۳۰۰ کلمه‌ای تقسیم می‌کنیم،
    به طوری که هر کلمه در دو پنجره ظاهر شود و با این کار برای هر فایل ممکن است چند بردار تولید شود
    نهایتا نزدیک‌ترین بردار آن به کوئری را به عنوان پاسخ آن در نظر می‌گیریم؛
    اما گویا به طور کلی از برت برای داده‌های بزرگ استفاده نمی‌شود
</div>

In [ ]:
def get_embed(part):
  encoding = tokenizer.encode_plus(
    part,
    add_special_tokens=True, # Add '[CLS]' and '[SEP]'
    return_token_type_ids=False,
    max_length = 500,
    truncation=True,
    return_attention_mask=True,
    return_tensors='pt',  # Return PyTorch tensors
  )
  encoding = {k:v.cuda() for k,v in encoding.items()}
  with torch.no_grad():
    out = model(**encoding)
  return out['pooler_output'].cpu().numpy()

In [ ]:
doc_vec = np.empty((0, 768))
doc_map = np.empty(0)

for index, doc in tqdm.tqdm(corpus.iterrows()):
  doc_split = doc['text'].split()
  doc_parts = [' '.join(doc_split[i:i + 300]) for i in range(0, len(doc_split) - 150, 150)]
  for part in doc_parts:
    doc_vec = np.append(doc_vec, get_embed(part), axis = 0)
    doc_map = np.append(doc_map, index)

3258it [06:04,  8.95it/s]


<div dir='rtl'>
    پیاده‌سازی روش سوم که
    tf-idf
    وزن‌دار برحسب
    pos-tag
    هاست در نوتبوکی رو فولدر
    method-3
    آمده‌است
</div>

# Document Retrieval

<div dir='rtl'>
    با استفاده از تابع نزدیک‌ترین همسایه‌ها در
    sklearn
    کلاسی برای اجرای الگوریتم
    KNN
    می‌سازیم که داک‌ها را
    fit
    کند و داک‌های نزدیک کوئری را حدس بزند.
    برای هر دو روش برت و
    tf-idf
    از
    knn
    استفاده می‌کنیم
</div>

In [60]:
class KNN_based_IR(BaseEstimator):
  def __init__(self,n_neighbors=1+10+10+10,metric='euclidean') -> None:
    self.n_neighbors = n_neighbors
    self.metric = metric
    self.nn = NearestNeighbors(n_neighbors=n_neighbors,metric=metric)
  def set_params(self,**kwargs):
    if 'n_neighbors' in kwargs:
      self.n_neighbors = kwargs['n_neighbors']
    if 'metric' in kwargs:
      self.metric = kwargs['metric']
    self.nn.set_params(**kwargs)
  def fit(self, X: np.array,y: None):
    self.nn.fit(X)
  def predict(self, X: np.array):
    distances, docIds = self.nn.kneighbors(X,n_neighbors=self.n_neighbors)
    scores = np.max(distances)-distances
    return scores, docIds

In [61]:
IR_system = KNN_based_IR()
IR_system.fit(vectorizer.transform(corpus.text),None)

In [ ]:
bert_knn = KNN_based_IR(80)
bert_knn.fit(doc_vec)

# IR Evaluation
Tailored for our multi-level Test Collection.

In [ ]:
def tf_knn_pred(knn):
  return knn.predict(vectorizer.transform(query))


preds = tf_knn_pred(IR_system)

<div dir='rtl'>
    برای تخمین برت برحسب
    knn
    باید تعداد بیشتری بردار نزدیک را به دست بیاوریم و بردارهای مربوط به یک داک یکسان را از روی آن حذف کنیم
    و فقط نزدیک‌ترین را باقی بگذاریم
</div>

In [ ]:
def bert_knn_pred(knn):
  bert_score = []
  bert_id = []
  mn = 100
  for q in tqdm.tqdm(query):
    score, oid = knn.predict(get_embed(q))
    score = score[0]
    doc_id = [doc_map[i] for i in oid[0]]
    n_score = []
    n_id = []
    for sc, id in zip(score, doc_id):
      if id not in n_id:
        n_id.append(id)
        n_score.append(sc)
    mn = min(mn, len(n_score[:31]))
    bert_score.append(n_score[:31])
    bert_id.append(n_id[:31])
  
  return (np.array(bert_score), np.array(bert_id).astype(int))


bert_pred = bert_knn_pred(bert_knn)

100%|██████████| 150/150 [00:06<00:00, 24.19it/s]


## Adapting IR output to our Test Collection

<div dir='rtl'>
    از تابع زیر برای اینکه بتوانیم خروجی بازیابی را تبدیل به حالتی برای ارزیابی کوئری‌ها بکنیم استفاده می‌کنیم.
    ما برای اینکه بتوانیم خروجی
    knn
    را تقریبا نزدیک به خروجی کوئری‌ها کنیم، تقریب زدیم که غیر از داک اصلی،
    از هر سطح نزدیکی به طور میانگین ۱۰ داک بازیابی می‌شود و با استفاده از آن ملاک‌ها را اندازه می گیریم
</div>

In [65]:
def adapt_IR_output_to_measure_input(IR_output: Tuple[np.array, np.array]):
  scores, docIds = IR_output
  return pd.DataFrame({'query_id': np.tile(query.index,(docIds.shape[1],1)).flatten(order='F').astype(str),
                       'doc_id':   docIds.flatten().astype(str),
                       'score':    scores.flatten()})

<div dir='rtl'>
    این تابع نیز برای بهبود عملکرد
    knn
    ایجاد شده است تا با تغییر متریک آن بتوانیم بازیابی بهتری با توجه به ملاک‌های ارزیابی داشته باشیم
</div>

In [ ]:
def knn_tuning(n, param, embed, pred_f, measure):
  score = -1
  best_p = -1
  for p in param:
    knn = KNN_based_IR(n)
    knn.set_params(metric = p)
    knn.fit(embed)
    val = measure(qrels.astype({'query_id':str,'doc_id':str}),pred_f(knn))
    if val > score:
      score = val
      best_p = p
  return best_p, score


param = ['cityblock', 'cosine', 'euclidean', 'l1', 'l2', 'manhattan'] # Metrics for sparse input

## MRR (Mean Reciprocal Rank)

<div dir='rtl'>
    برای استفاده از این اندازه‌گیری، باید محل مرتبط‌ترین داک هر کوئری را بین لیست داک‌های بازیابی شده بیابیم و معکوس آن را حساب کنیم و سپس میانگین بگیریم.
</div>

In [36]:
MRR = IRm.measures.MRR()
def mrr_measure(qrels, ret):
  ret = adapt_IR_output_to_measure_input(ret)
  return MRR.calc_aggregate(qrels[qrels.relevance == 4], ret)
# mrr_scorer = make_scorer(mrr)

In [ ]:
mrr_measure(qrels.astype({'query_id':str,'doc_id':str}),preds)

0.11047063325391496

In [ ]:
knn_tuning(31, param, vectorizer.transform(corpus.text), tf_knn_pred, mrr_measure)

('cosine', 0.11664736491005835)

In [ ]:
mrr_measure(qrels.astype({'query_id':str,'doc_id':str}),bert_pred)

0.06931617823372208

In [ ]:
knn_tuning(80, param, doc_vec, bert_knn_pred, mrr_measure)

100%|██████████| 150/150 [00:05<00:00, 25.57it/s]


('cosine', 0.06982707206877474)

## MAP (Mean Average Precision)

In [37]:
def map_measure(qrels, ret):
  ret = adapt_IR_output_to_measure_input(ret)
  return np.mean([IRm.measures.AP(rel=level).\
                    calc_aggregate(qrels[qrels.relevance == level], ret) for level in range(1,4+1)])

# map_scorer = make_scorer(map)

In [ ]:
map_measure(qrels.astype({'query_id':str,'doc_id':str}),preds)

0.06630537139767945

In [ ]:
knn_tuning(31, param, vectorizer.transform(corpus.text), tf_knn_pred, map_measure)

('cosine', 0.06931719832816999)

In [ ]:
map_measure(qrels.astype({'query_id':str,'doc_id':str}),bert_pred)

0.048315030270106925

In [ ]:
knn_tuning(80, param, doc_vec, bert_knn_pred, map_measure)

100%|██████████| 150/150 [00:05<00:00, 25.91it/s]


('cityblock', 0.04838834768432319)

## P@K

<div dir='rtl'>
    با توجه به اینکه در کوئری‌ها ما رنکینگ نداریم، برای محاسبه این معیار
    precision
    را در
    i
    دسته اول حساب می‌کنیم و سپس میانگین آن‌ها را برمی‌گردانیم
</div>

In [38]:
def p_measure(qrels, ret):
  ret = adapt_IR_output_to_measure_input(ret)
  return np.mean([IRm.measures.P(cutoff=k, rel=level).\
                    calc_aggregate(qrels[qrels.relevance == level], ret)\
                  for k,level in zip([1,11,21,31],range(1,4+1))])

In [ ]:
p_measure(qrels.astype({'query_id':str,'doc_id':str}),preds)

0.033872596937113045

In [ ]:
knn_tuning(31, param, vectorizer.transform(corpus.text), tf_knn_pred, p_measure)

('cosine', 0.03418284224735836)

In [ ]:
p_measure(qrels.astype({'query_id':str,'doc_id':str}),bert_pred)

0.026597076758367057

In [ ]:
knn_tuning(80, param, doc_vec, bert_knn_pred, p_measure)

100%|██████████| 150/150 [00:05<00:00, 26.06it/s]


('cosine', 0.02688893543732251)

<div dir='rtl'>
    بنظر می‌رسد
    tuning
    های انجام شده تغییر ناچیزی در امتیازهای ما داشته‌اند و نتیجه را بهبود چندانی نداده‌اند
</div>

# Pipeline Definition

In [82]:
param_grid = {
    'embedding__ngram_range': [(1,2),(1,3),(1,4)],
    'embedding__max_features': range(100000,1000,-10000),
    'retrieval__n_neighbors': range(10,100,10),
    'retrieval__metric': ['cityblock', 'cosine', 'euclidean', 'l1', 'l2', 'manhattan']
}
measures = {
    'MRR' : mrr_measure,
    'MAP' : map_measure,
    'P@K' : p_measure
}

In [83]:
cache = mkdtemp()
pipeline = Pipeline([('embedding',vectorizer),
                     ('retrieval',IR_system)],memory=cache)

In [84]:
results = []
index = []
for params in tqdm.tqdm(ParameterGrid(param_grid)):
  pipeline.set_params(**params)
  pipeline.fit(corpus.text,None)
  preds = pipeline.predict(query)
  results.append(tuple(m(qrels.astype({'query_id':str,'doc_id':str}),preds) for _,m in measures.items()))
  index.append(tuple(p for _,p in params.items()))
results = pd.DataFrame(results,columns=list(measures.keys()),
                       index=pd.MultiIndex.from_tuples(index,names=list(param_grid.keys())))

  0%|          | 0/1620 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/sklearn/pipeline.py:355: UserWarning: Persisting input arguments took 16.47s to run.
If this happens often in your code, it can cause performance problems 
(results will be correct in all cases). 
The reason for this is probably some large input arguments for a wrapped
 function (e.g. large strings).
THIS IS A JOBLIB ISSUE. If you can, kindly provide the joblib's team with an
 example so that they can fix the problem.
  **fit_params_steps[name],
  3%|▎         | 54/1620 [04:48<2:12:58,  5.09s/it]/usr/local/lib/python3.7/dist-packages/sklearn/pipeline.py:355: UserWarning: Persisting input arguments took 28.41s to run.
If this happens often in your code, it can cause performance problems 
(results will be correct in all cases). 
The reason for this is probably some large input arguments for a wrapped
 function (e.g. large strings).
THIS IS A JOBLIB ISSUE. If you can, kindly provide the joblib's team with an
 e

In [78]:
pd.DataFrame(results,columns=list(measures.keys()),
             index=pd.MultiIndex.from_tuples(index,names=list(param_grid.keys())))

MRR  ...       P@K
embedding__max_features embedding__ngram_range retrieval__n_neighbors retrieval__metric            ...          
500                     (1, 2)                 cityblock              10                 0.010000  ...  0.003008
                                                                      20                 0.010351  ...  0.003531
                                                                      30                 0.010618  ...  0.003584
                                                                      40                 0.010618  ...  0.003584
                                                                      50                 0.010756  ...  0.003584
                                                                      60                 0.010875  ...  0.003584
                                                                      70                 0.010980  ...  0.003584
                                                                      80                 0.011154  ...  0.003584
                                                                      90                 0.011154  ...  0.003584
                                               cosine                 10                 0.111034  ...  0.025689
                                                                      20                 0.114183  ...  0.033223
                                                                      30                 0.116647  ...  0.034183
                                                                      40                 0.117170  ...  0.034183
                                                                      50                 0.117584  ...  0.034183

[14 rows x 3 columns]

In [93]:
results.MRR

embedding__ngram_range  embedding__max_features  retrieval__n_neighbors  retrieval__metric
100000                  (1, 2)                   cityblock               10                   0.000000
                                                                         20                   0.000000
                                                                         30                   0.000000
                                                                         40                   0.000190
                                                                         50                   0.000640
                                                                                                ...   
10000                   (1, 4)                   manhattan               50                   0.001200
                                                                         60                   0.001430
                                                                         70          

In [98]:
results.idxmax()

MRR    (60000, (1, 4), cosine, 80)
MAP        (50000, (1, 2), l2, 90)
P@K    (20000, (1, 3), cosine, 30)
dtype: object

In [99]:
results.max()

MRR    0.560743
MAP    0.330364
P@K    0.110606
dtype: float64